## Set up the environment

In [14]:
import sagemaker as sage
from sagemaker import get_execution_role

MODEL_NAME = "[Specify a name for your model e.g. last segment of Model Arn]"

# S3 paths for input and output data 
# Ensure the buckets are in the same region as subscribed model
S3_INPUT_PATH = "s3://[INPUT_BUCKET_NAME]/[INPUT_FILE]"
S3_OUTPUT_PATH = "s3://[OUTPUT_BUCKET_NAME]/output"
S3_OUTPUT_FILE = S3_OUTPUT_PATH + "/[INPUT_FILE].out"

# Specify one of the supported real time instances for the subscribed model
INFERENCE_INSTANCE = "ml.p2.xlarge"

# Specify one of the supported batch transform instances for the subscribed model
BATCH_INSTANCE = "ml.m5.large"

# ROLE = get_execution_role()
# or specify role arn
# ensure the role has access to S3 buckets specified above
ROLE = "arn:aws:iam::[your aws account]:role/[your sagemaker role name]"

MODEL_PACKAGE_ARN ="arn:aws:sagemaker:[region]:[your aws account id]:model-package/[Model name]"

sagemaker_session = sage.Session()


arn:aws:sagemaker:us-east-2:390946248509:model-package/point-of-compromise-fraud-detection


## Realtime Inference

In [13]:
from sagemaker import ModelPackage
model = ModelPackage(name=MODEL_NAME,
                     role=ROLE,
                     model_package_arn=MODEL_PACKAGE_ARN,
                     sagemaker_session=sagemaker_session)

In [15]:
endpoint_name=MODEL_NAME + "-edp"

In [16]:
predictor = model.deploy(1, INFERENCE_INSTANCE, endpoint_name=endpoint_name)

---------------!

In [ ]:
# connect to the endpoint
predictor = sage.Predictor(
    endpoint_name,
    sagemaker_session=sagemaker_session
)

In [ ]:
import s3fs, json
fs = s3fs.S3FileSystem()

with fs.open(S3_INPUT_PATH) as f:    
    docByteArr = f.read()
    response = predictor.predict(docByteArr)
    result=response.decode('utf-8')
    print(json.loads(result))

In [19]:
# Delete endpoint
predictor.delete_endpoint()

AttributeError: 'NoneType' object has no attribute 'delete_endpoint'

## Batch Transform Job

In [17]:
transformer = sage.transformer.Transformer(
    base_transform_job_name='batch-transform',
    model_name=MODEL_NAME,
    instance_count=1,
    instance_type=BATCH_INSTANCE,
    output_path=S3_OUTPUT_PATH,
    sagemaker_session=sagemaker_session
)


In [18]:
transformer.transform(
    S3_INPUT_PATH
)

# wait until prediction job is completed
transformer.wait()


........................
2020-10-16T19:13:12.081:[sagemaker logs]: MaxConcurrentTransforms=1, MaxPayloadInMB=6, BatchStrategy=MULTI_RECORD
 * Serving Flask app "point-of-compromise-fraud-detection" (lazy loading)
 * Environment: PRODUCTION
 * Debug mode: off
 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
169.254.255.130 - - [16/Oct/2020 19:13:12] "#033GET /ping HTTP/1.1#033" 200 -
169.254.255.130 - - [16/Oct/2020 19:13:12] "#033GET /execution-parameters HTTP/1.1#033" 404 -
169.254.255.130 - - [16/Oct/2020 19:13:12] "#033POST /invocations HTTP/1.1#033" 200 -
 * Serving Flask app "point-of-compromise-fraud-detection" (lazy loading)
 * Environment: PRODUCTION
 * Debug mode: off
 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
169.254.255.130 - - [16/Oct/2020 19:13:12] "#033GET /ping HTTP/1.1#033" 200 -
169.254.255.130 - - [16/Oct/2020 19:13:12] "#033GET /execution-parameters HTTP/1.1#033" 404 -
169.254.255.130 - - [16/Oct/2020 19:13:12] "#033POST /invocations HTTP/1.1#

#### Inspect the Batch Transform Output in S3

In [20]:
s3_client = sage.s3.S3Downloader
s3_client.read_file(S3_OUTPUT_FILE)

'[{"mid_zip5":"526046724433_60601","score":0.68380951},{"mid_zip5":"527583689359_92101","score":0.9800017},{"mid_zip5":"552121103103_63101","score":0.72471319},{"mid_zip5":"139243061176_94203","score":0.78034101},{"mid_zip5":"550279697705_92101","score":0.52234835},{"mid_zip5":"917452601856_30301","score":0.27969479},{"mid_zip5":"922278518399_95101","score":0.516166},{"mid_zip5":"781868299537_30301","score":0.18081539},{"mid_zip5":"593471958486_68046","score":0.44803247},{"mid_zip5":"582418463690_60601","score":0.93796972}]'